### Text Extraction from PDF (Contained Images Only)

In [1]:
# ocr_to_text , list of pdf that failed to be converted to text , need to user OCR approach instead (image only)

import pandas as pd
ocr_to_text=pd.read_csv('/Users/izard/Documents/ocr_to_text.csv') # open the filtered un-readable pdf (initially via Tika)
ocr_to_text

,Document,Document_Text
0,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN
1,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN
2,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN
3,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN
4,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN
...,...,...
2100,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN
2101,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN
2102,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN
2103,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN


In [ ]:
# find image in pdf pages and save those image in folder that using the same filename 

import fitz #pymupdf
import io
from PIL import Image

docs = ocr_to_text['Document'].to_list()
for doc in docs:
    os.chdir('/Users/izard/Documents/pdf/')
    os.mkdir(doc.replace('https://efs.kehakiman.gov.my/EFSWeb/DocDownloader.aspx?DocumentID=','').replace('&Inline=true',''))
    
    output_folder=str(doc.replace('https://efs.kehakiman.gov.my/EFSWeb/DocDownloader.aspx?DocumentID=', '').replace('&Inline=true',''))
    
    try:    
        # Open the PDF file
        pdf_file = fitz.open(doc.replace('https://efs.kehakiman.gov.my/EFSWeb/DocDownloader.aspx?DocumentID=','').replace('&Inline=true','.pdf'))
        
        os.chdir('/Users/izard/Documents/pdf/'+output_folder)
        
        # Iterate over PDF pages
        for page_index in range(len(pdf_file)):
            page = pdf_file[page_index]
            pix = page.get_pixmap()  # render page to an image
            pix.save("page-%i.png" % page.number)  # store image as a PNG
    except:
        pass
        

In [2]:
# function to convert images in folders into text

import os
from PIL import Image
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files (x86)\Tesseract-OCR\tesseract.exe"

def image_to_text(x):
    # Get the list of image files in the PDF folder
    images = os.listdir('/Users/izard/Documents/pdf/' + x.replace('https://efs.kehakiman.gov.my/EFSWeb/DocDownloader.aspx?DocumentID=', '').replace('&Inline=true', ''))
    
    # Initialize an empty string to store the text
    text = ''
    
    # Loop through the images and append the text to the string
    for image in images:
        img = Image.open('/Users/izard/Documents/pdf/' + x.replace('https://efs.kehakiman.gov.my/EFSWeb/DocDownloader.aspx?DocumentID=', '').replace('&Inline=true', '') + '/' + image)
        text = text + ' ' + pytesseract.image_to_string(img, config='--psm 13',lang='eng+msa') #lang='eng+msa' for english and malay
        
        '''
        Page segmentation modes:
        0    Orientation and script detection (OSD) only.
        1    Automatic page segmentation with OSD.
        2    Automatic page segmentation, but no OSD, or OCR.
        3    Fully automatic page segmentation, but no OSD. (Default)
        4    Assume a single column of text of variable sizes.
        5    Assume a single uniform block of vertically aligned text.
        6    Assume a single uniform block of text.
        7    Treat the image as a single text line.
        8    Treat the image as a single word.
        9    Treat the image as a single word in a circle.
        10   Treat the image as a single character.
        11   Sparse text. Find as much text as possible in no particular order.
        12   Sparse text with OSD.
        13   Raw line. Treat the image as a single text line,bypassing hacks that are Tesseract-specific
        '''
    
    # Return the text
    return text


In [3]:
# apply the function in pandas df to obtain the extracted text from images in the folders

try:
    ocr_to_text['Document_Text'] = ocr_to_text['Document'].apply(lambda x: image_to_text(x))
except:
    pass

In [ ]:
ocr_to_text.to_csv('/Users/izard/Documents/ocr_to_text_filtered_config_1.csv', index=False)